### Additional analysis (2): Bayesian analysis

Based on the dataset, we can define a **Bayesian logistic regression model** that uses the **IgA-related variables** to estimate the probability of an individual being classified as **insecurely attached**. This approach is conceptually aligned with your previous findings and avoids potential confounds from hormonal measures like cortisol.


### **Bayesian Model Specification**

Let $y_i \in \{0, 1\}$ be the binary attachment label for individual $i$, where 1 = insecure, and 0 = secure.

We model the probability $p_i$ of insecure attachment via a logistic function:

$$
y_i \sim \text{Bernoulli}(p_i)
$$

$$
\text{logit}(p_i) = \alpha + \beta_1 \cdot \text{IgA}_{\text{morning}, i} + \beta_2 \cdot \text{IgA}_{\text{afternoon}, i} + \beta_3 \cdot \text{IgA}_{\text{difference}, i}
$$

Where:

* $\alpha \sim \mathcal{N}(0, 2)$ is the intercept,
* $\beta_j \sim \mathcal{N}(0, 2)$ are priors for the IgA coefficients.

In [ ]:
import pymc as pm
import arviz as az

# Encode attachment
df_data_plot['Attachment_binary'] = df_data_plot['Attachment'].map({'Secure': 0, 'Insecure': 1})

# Select predictors
X = df_data_plot[['sIgA_Average_Morning', 'sIgA_Average_Afternoon', 'sIgA_Average_Difference']]
y = df_data_plot['Attachment_binary'].values

# Normalize predictors
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
iga_morning = X_scaled[:, 0]
iga_afternoon = X_scaled[:, 1]
iga_diff = X_scaled[:, 2]

# Define Bayesian logistic regression model
with pm.Model() as iga_model:
    alpha = pm.Normal("alpha", mu=0, sigma=2)
    beta_morning = pm.Normal("beta_siga_morning", mu=0, sigma=2)
    beta_afternoon = pm.Normal("beta_siga_afternoon", mu=0, sigma=2)
    beta_diff = pm.Normal("beta_siga_diff", mu=0, sigma=2)

    # Linear predictor
    mu = (alpha +
          beta_morning * iga_morning +
          beta_afternoon * iga_afternoon +
          beta_diff * iga_diff)

    theta = pm.Deterministic("theta", pm.math.sigmoid(mu))
    y_obs = pm.Bernoulli("y_obs", p=theta, observed=y)

    # Posterior sampling
    trace = pm.sample(
        draws=1000,
        tune=1000,
        chains=2,
        cores=1,
        init="adapt_diag",
        target_accept=0.9,
        return_inferencedata=True
    )

# Posterior summary
az.summary(trace, hdi_prob=0.94)

In [ ]:
az.plot_forest(trace, var_names=["beta_siga_morning", "beta_siga_afternoon", "beta_siga_diff"], combined=True)

Based on the results of the Bayesian logistic regression model that uses only IgA-related variables—namely, **Average Morning IgA**, **Average Afternoon IgA**, and **Average IgA Difference**—we can derive a nuanced understanding of how these physiological markers relate to the probability of **insecure attachment**. The model posterior distributions and the 94% Highest Density Intervals (HDIs) provide insight into the direction, strength, and uncertainty of each predictor’s contribution, while the estimated probabilities (`theta[i]`) allow us to examine how the model performs at the individual level.

---

### **Posterior Coefficients and Uncertainty (HDI Analysis)**

The HDI figure and numerical table show that **all three predictors exhibit wide 94% HDIs**, and none of the posterior intervals exclude zero. This suggests considerable uncertainty in their effects, likely driven by sample size limitations and variability within the data. Nevertheless, the **direction and magnitude of the posterior means** offer important interpretive guidance.

The coefficient for **Average IgA Morning** (posterior mean = -1.10) has the most negative estimate, with an HDI ranging from approximately -3.47 to +1.63. This suggests a tendency for higher morning IgA values to be associated with a lower probability of insecure attachment. The direction of the effect is theoretically consistent with prior observations that secure individuals tend to exhibit more stable and elevated IgA levels, possibly reflecting immune regulation aligned with psychological security.

Similarly, **Average IgA Afternoon** also has a negative posterior mean (–0.38), but with a narrower magnitude and an HDI still wide (–2.54 to +1.76). This weaker and more uncertain signal may reflect the more variable nature of afternoon IgA across individuals or reduced relevance of this timepoint in capturing immunological correlates of stress regulation.

In contrast, the coefficient for **Average IgA Difference** is positive (posterior mean = +0.78), with an HDI spanning from –1.88 to +3.22. This aligns with theoretical expectations that greater diurnal variation in IgA may be linked to physiological reactivity—often seen in individuals with insecure attachment. While uncertainty remains high, the positive direction is consistent with earlier findings from clustering and correlation analyses.

The model suggests a **pattern where stable, higher IgA—particularly in the morning—is associated with security**, whereas **variability across the day may signal dysregulation** related to insecure attachment. However, due to wide posterior intervals, these should be interpreted as trends rather than definitive associations.

---

### **Model Predictions: Individual Probability Estimates (`theta`)**

The posterior estimates for the predicted probabilities (`theta[i]`) range broadly, from values as low as 0.01 to as high as 0.88. This range demonstrates that the model successfully captures variation across individuals, with some cases clearly leaning toward secure classification (e.g., `theta[2] = 0.018`, `theta[10] = 0.031`) and others with high predicted probabilities of insecure attachment (e.g., `theta[7] = 0.624`, `theta[19] = 0.564`, `theta[21] = 0.604`).

Notably, individuals with high predicted values tend to fall within the posterior tails where model uncertainty is still relatively low—this suggests that, although the population-level coefficients are not definitive, the model has sufficient discriminatory power for **case-level probabilistic classification**. The credible intervals for individual predictions are generally tight, indicating that the model is **well-calibrated despite small data** and that posterior variance is not driven by poor convergence.

---

### **Model Behavior and Relevance to Previous Findings**

These results mirror and refine previous observations made from exploratory clustering and visualizations. In prior analyses, secure individuals tended to group tightly in low-variance clusters of IgA profiles, while insecure individuals displayed more physiological dispersion. The current model extends that pattern probabilistically: the more an individual's IgA profile aligns with stability (e.g., higher morning values, smaller difference), the lower their modeled probability of insecurity.

The Bayesian framework enhances this narrative by providing **uncertainty quantification**, avoiding overfitting, and delivering interpretable intervals for each effect. While classic logistic regression might overstate effect sizes or confidence, this approach respects the small-n nature of the data while still offering mechanistic insight.

---

### **Limitations and Caution**

The primary limitation remains the **small sample size** and the resulting **wide posterior intervals**. None of the HDIs exclude zero, which indicates that we cannot, with 94% certainty, claim these effects are directionally different from zero. Furthermore, potential multicollinearity among IgA measures (particularly morning and afternoon values) might obscure the individual contribution of each variable. Future models could benefit from either dimensionality reduction or hierarchical modeling to account for repeated patterns or inter-subject variability.

---

### **Conclusion**

This Bayesian model suggests that higher IgA levels—particularly in the morning—are probabilistically associated with lower odds of insecure attachment, while greater diurnal IgA variability may be associated with greater risk. Although uncertainty remains high, the direction of effects and the model's individualized predictions provide converging evidence that IgA can reflect attachment-relevant physiological organization. The model complements earlier exploratory and machine learning results, offering a probabilistic, interpretable, and theoretically consistent bridge between immune data and attachment classification.
